### Working With ASDF Files

#### Outline

- Read a file
- Show the contents of an ASDF file
- Search for an attribute in an ASDF file
- Accessing metadata and data
- Modifying and saving files
- Exercise
- Adding History items

#### Reading an ASDF file

The Python ASDF library is a standalone package distributed through PyPi and conda-forge.

In [3]:
import asdf

To open a file use the **open** function. It is useful to look up the keyword arguments it accepts, there are many options specifying how a file should be opened or validated during opening. For this example we will use the default behavior and look at the object.

In [5]:
af = asdf.open('r002_assign_wcs.asdf')

#### Getting information about a file

There are two functions that allow introspecting a file, *info* and *search*. They are available as methods on the AsdFile object or on the command line. Both are configurable through multiple parameters.

**The *info* method shows the contents of a file**. 

In [11]:
af.info()

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): The ASDF Developers
│ ├─homepage (str): http://github.com/asdf-format/asdf
│ ├─name (str): asdf
│ └─version (str): 2.11.2.dev13+gf9aeb247
├─history (dict)
│ └─extensions (list)
│   ├─[0] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/transform/extensions/transform-1.5.0
│   │ └─software (Software) ...
│   ├─[1] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension.BuiltinExtension
│   │ └─software (Software) ...
│   ├─[2] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/astronomy/gwcs/extensions/gwcs-1.0.0
│   │ └─software (Software) ...
│   ├─[3] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://stsci.edu/datamodels/roman/extensions/datamode

The asdf library has search capabilities. A file can be search for an attribute by name, type or value.

In [12]:
af.search('wcs')

root (AsdfObject)
└─roman (WfiImage) # The schema for WFI Level 2 images.

  └─meta (dict)
    ├─cal_step (CalStep) # Calibration Status
    │ └─assign_wcs (str): COMPLETE # Assign World Coordinate System
    ├─wcs (WCS)
    └─wcsinfo (Wcsinfo) # WCS parameters

In [14]:
af.search(value='WFI')

root (AsdfObject)
└─roman (WfiImage) # The schema for WFI Level 2 images.

  ├─meta (dict)
  │ ├─aperture (Aperture) # Aperture information
  │ │ └─name (str): WFI_CEN # PRD science aperture used
  │ ├─exposure (Exposure) # Exposure information

  │ │ └─type (str): WFI_GRISM
  │ ├─filename (str): r0000201001001001002_01101_0001_WFI07.asdf
  │ └─instrument (WfiMode) # WFI observing configuration

  │   ├─detector (str): WFI07
  │   └─name (str): WFI # Instrument used to acquire the data
  ├─cal_logs (CalLogs) # Calibration log messages
  │ ├─10 (str): 2022-06-21T10:54:12.506Z :: stpipe.ExposurePipeline.rampfit :: DEBUG :: Instrument: WFI
  │ ├─29 (str): 2022-06-21T10:50:40.770Z :: stpipe.ExposurePipeline :: INFO :: Step Exposure (truncated)
  │ ├─31 (str): 2022-06-21T10:50:40.779Z :: stpipe.ExposurePipeline :: INFO :: First argumen (truncated)
  │ ├─33 (str): 2022-06-21T10:50:40.870Z :: stpipe.ExposurePipeline :: DEBUG :: Exposure Pro (truncated)
  │ ├─79 (str): 2022-06-21T10:54:12.506Z

In [13]:
from gwcs import WCS

af.search(type=WCS)

root (AsdfObject)
└─roman (WfiImage) # The schema for WFI Level 2 images.

  └─meta (dict)
    └─wcs (WCS)

#### Accessing and Modifying a file

In [16]:
w = af['roman']['meta']['wcs']
print(w)

  From     Transform  
-------- -------------
detector CompoundModel
    v2v3      v23tosky
   world          None


In [18]:
af['roman']['data']

array([[13.169418 , 11.421084 , 31.831703 , ..., 28.526772 , 15.065651 ,
        16.435959 ],
       [ 2.0519984, 22.071646 , 28.865814 , ..., 10.424057 , 10.968319 ,
        50.12     ],
       [17.03684  , 14.658701 , 27.407093 , ..., 11.328522 , 16.260082 ,
        12.559191 ],
       ...,
       [26.335503 , 16.206621 , 12.851121 , ..., 21.996883 , 22.182444 ,
         8.487001 ],
       [18.034378 , 16.855669 ,  8.183829 , ..., 29.054497 , 16.021288 ,
         0.3593996],
       [42.71249  , -4.2684755, 40.994675 , ..., 45.805332 , 14.370688 ,
        10.289682 ]], dtype=float32)

Let's modify the first element of the array. By default ASDF files are opened in *read only* mode.

In [19]:
af['roman']['data'][0,0]=0

ValueError: assignment destination is read-only

In [34]:
af = asdf.open('r002_assign_wcs.asdf', mode='rw')
af['roman']['data'][0, 0]=0
#af.write_to('r002_modified.asdf')
af.close()
af = asdf.open('r002_assign_wcs.asdf')
af['roman']['data']
af.write_to('todel.asdf')

ValueError: Block '8' not found.

#### Adding History items

ASDF files have a *History* field which stores information abou tthe software used to reate the file, including all custom extensions.

In [23]:
af['history']

{'extensions': [{'extension_class': 'asdf.extension._manifest.ManifestExtension',
   'extension_uri': 'asdf://asdf-format.org/transform/extensions/transform-1.5.0',
   'software': {'name': 'asdf-astropy', 'version': '0.2.1'}},
  {'extension_class': 'asdf.extension.BuiltinExtension',
   'software': {'name': 'asdf', 'version': '2.11.2.dev13+gf9aeb247'}},
  {'extension_class': 'asdf.extension._manifest.ManifestExtension',
   'extension_uri': 'asdf://asdf-format.org/astronomy/gwcs/extensions/gwcs-1.0.0',
   'software': {'name': 'gwcs', 'version': '0.18.1'}},
  {'extension_class': 'asdf.extension._manifest.ManifestExtension',
   'extension_uri': 'asdf://stsci.edu/datamodels/roman/extensions/datamodels-1.0',
   'software': {'name': 'roman-datamodels', 'version': '0.12.2'}},
  {'extension_class': 'asdf.extension._manifest.ManifestExtension',
   'extension_uri': 'asdf://asdf-format.org/core/extensions/core-1.5.0',
   'software': {'name': 'asdf-astropy', 'version': '0.2.1'}},
  {'extension_clas

In [25]:
af.add_history_entry("This file was reflects the state of the ROman pipeline on Jun 20, 2022")

In [27]:
af['history']

{'extensions': [{'extension_class': 'asdf.extension._manifest.ManifestExtension',
   'extension_uri': 'asdf://asdf-format.org/transform/extensions/transform-1.5.0',
   'software': {'name': 'asdf-astropy', 'version': '0.2.1'}},
  {'extension_class': 'asdf.extension.BuiltinExtension',
   'software': {'name': 'asdf', 'version': '2.11.2.dev13+gf9aeb247'}},
  {'extension_class': 'asdf.extension._manifest.ManifestExtension',
   'extension_uri': 'asdf://asdf-format.org/astronomy/gwcs/extensions/gwcs-1.0.0',
   'software': {'name': 'gwcs', 'version': '0.18.1'}},
  {'extension_class': 'asdf.extension._manifest.ManifestExtension',
   'extension_uri': 'asdf://stsci.edu/datamodels/roman/extensions/datamodels-1.0',
   'software': {'name': 'roman-datamodels', 'version': '0.12.2'}},
  {'extension_class': 'asdf.extension._manifest.ManifestExtension',
   'extension_uri': 'asdf://asdf-format.org/core/extensions/core-1.5.0',
   'software': {'name': 'asdf-astropy', 'version': '0.2.1'}},
  {'extension_clas